Prediction of best team in football

In [ ]:
# import the modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


### Load the dataset

- Load the train data and using all your knowledge of pandas try to explore the different statistical properties of the dataset.

In [ ]:
# read the dataset and extract the features and target separately
train = pd.read_csv('../input/fifa-best-team/train.csv',)
test = pd.read_csv('../input/fifa-best-team/test.csv')
ss = pd.read_csv('../input/fifa-best-team/sample_submission.csv')

In [ ]:
#seeing the glance of train data
train.head()

In [ ]:
test.head()

In [ ]:
ss.head()

In [ ]:
print(f'Train columns has {train.shape[0]} observation and {train.shape[1]} columns')
print(f'Train columns has {test.shape[0]} observation and {test.shape[1]} columns')

In [ ]:
train.columns

In [ ]:
train.dtypes

In [ ]:
train.select_dtypes(include='number').columns

In [ ]:
train.select_dtypes(include='object').columns

In [ ]:
train.drop(columns=['Id'],inplace=True)

In [ ]:
train.info()

### Visualize the data

- Check for the categorical & continuous features. 
- Check out the best plots for plotting between categorical target and continuous features and try making some inferences from these plots.
- Check for the correlation between the features

In [ ]:
train.groupby(['Name','Age','Club','Nationality'])['Nationality'].count()

In [ ]:
train['Club'].value_counts()

In [ ]:
train['Nationality'].value_counts()[:20]

In [ ]:
train['Position'].value_counts()

as we see CB is the highest frequency positioned in our data that is Center Back,
Which plays very important role in game. let's make an null hypothesis that as CB is higher 
value in data. 

In [ ]:
postion_n_value = train.groupby(['Position'])['Value (M)'].mean().sort_values(ascending=False)

In [ ]:
postion_n_value.plot(kind='barh')
plt.title('Position Vs Mean of Value ')
plt.show()

LW and RW are the most value position in data 

In [ ]:
train.groupby(['Nationality'])['Value (M)'].mean()[:20].sort_values(ascending=False)

We can see that the mean of for the values is highest for algerian country lets see the who are the players
From algeria taking most of value

In [ ]:
train[train['Nationality']=='Algeria']

In [ ]:
train['Age'].value_counts().plot(kind='bar')

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(train['Age'])

age variable is slightly postively skewed toward right

In [ ]:
sns.scatterplot(y= train['Value (M)'],x=train['Age'])
plt.title('Value Vs Age Scatter plot',fontSize=15)
plt.show()

In [ ]:
sns.distplot(train['Value (M)'])
plt.title('Distribution plot of Value ',fontSize=15)
plt.show()

value observation is heavily skewed towards right

In [ ]:
sns.boxplot((train['Value (M)']))
plt.title('box plot of Value ',fontSize=15)
plt.show()

From Box plot We can say that the most of observations in value outliers

In [ ]:
sns.distplot(np.log1p(train['Value (M)']))
plt.title('Distribution plot of Value ',fontSize=15)
plt.show()

In [ ]:
sns.boxplot(np.log1p(train['Value (M)']))
plt.title('box plot of Value ',fontSize=15)
plt.show()

In [ ]:
sns.distplot(train['Overall'])

In [ ]:
sns.distplot(train['Potential'])

In [ ]:
sns.distplot(train['Wage (M)'])

In [ ]:
train.describe()

In [ ]:
train.corr()

In [ ]:
train.columns

In [ ]:
sns.pairplot(train[['Overall', 'Potential',
       'Value (M)', 'Wage (M)']])

Seeing the pair plot , we can infer that the value is exponentially increases as the Overall and Potential is changes. And We can see their is linear relationship between the value and wage.

In [ ]:
train.head()

In [ ]:
train.groupby(train['Club'])['Value (M)'].mean().sort_values(ascending=False)[:15].plot(kind='barh',stacked=True)
plt.title('Club Vs Value Mean Barplot',fontSize=15,)
plt.show()

some of team in plot are very high mean of value as compare to others.

In [ ]:
import category_encoders as ce

encoder=ce.sum_coding.SumEncoder(cols='Position',handle_unknown='return_nan',return_df=True)




In [ ]:
train.head()

In [ ]:
position_encoded = encoder.fit_transform(train)

In [ ]:
position_encoded

In [ ]:
position_encoded.isnull().sum()

In [ ]:
X = position_encoded.drop(columns=['Name','Nationality','Club','Value (M)'])

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poloy_feat  = PolynomialFeatures(degree=2)
X =poloy_feat.fit_transform(X)


In [ ]:
y = position_encoded.loc[:,['Value (M)']]
y.head()

### Model building

- Separate the features and target and then split the train data into train and validation set.
- Now let's come to the actual task, using linear regression, predict the `Value (M)`. 
- Try improving upon the `r2_score` (R-Square) using different parameters that give the best score. You can use higher degree [Polynomial Features of sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) to improve the model prediction. 



In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
# Code Starts here

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

linReg = LinearRegression()

linReg.fit(X_train,y_train)

y_pred = linReg.predict(X_test)




# Code ends here

In [ ]:
mean_squared_error(y_test,y_pred)

In [ ]:
r2_score(y_test,y_pred)

### Prediction on the test data and creating the sample submission file.

- Load the test data and store the `Id` column in a separate variable.
- Perform the same operations on the test data that you have performed on the train data.
- Create the submission file as a `csv` file consisting of the `Id` column from the test data and your prediction as the second column.

In [ ]:
test.shape

In [ ]:
# Code Starts here




import category_encoders as ce

ncoder = ce.sum_coding.SumEncoder(cols='Position',handle_unknown='return_nan',return_df=True)

position_encoder1 = ncoder.fit_transform(test)


# Code ends here

In [ ]:
position_encoder1.drop(columns=['Id','Name','Nationality','Club'],inplace=True)

In [ ]:
position_encoder1.shape

In [ ]:
test_proc =poloy_feat.fit_transform(position_encoder1)

In [ ]:
lin_reg_preds = linReg.predict(test_proc)

In [ ]:
test_proc.shape

In [ ]:
new_frame = pd.DataFrame(index=test['Id'])

In [ ]:
new_frame['Value (M)'] = lin_reg_preds

In [ ]:
from IPython.display import HTML


new_frame.to_csv('submission.csv')

def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='submission.csv')